In [1]:
import numpy as np
from numba import jit

import plotly.express as px
import plotly.graph_objects as go

from utils import cartesian_product

In [2]:
max_wavevector_component_harmonic = 5
num_quasiwavevector_components = 99

In [3]:
wavevector_component_harmonics = np.arange(-max_wavevector_component_harmonic, max_wavevector_component_harmonic + 1)
wavevector_harmonics = cartesian_product(*(wavevector_component_harmonics,)*2)

position_components = np.linspace(-1/2, 1/2, wavevector_component_harmonics.size) # [spacing]

def wavevector_harmonic_to_index(wavevector_harmonic):
    return tuple(np.array(wavevector_harmonic) + np.full(2, max_wavevector_component_harmonic))

wavevector_harmonics.shape

(121, 2)

In [5]:
quasiwavevector_components = np.linspace(-1, 1, num_quasiwavevector_components) # [wavenumber]
quasiwavevectors = cartesian_product(*(quasiwavevector_components,)*2)

zero_quasiwavevector_index = (quasiwavevector_components.size - 1) // 2
max_quasiwavevector_index = -1

quasiwavevectors.shape

(9801, 2)

In [6]:
def compute_potential(square_depth, stripe_depth, superlattice_phase):
    intensity = np.zeros((wavevector_component_harmonics.size,)*2, dtype=complex)

    intensity[wavevector_harmonic_to_index((0, 0))]       = (1/4)*square_depth + (1/2)*stripe_depth
    for harmonic in ((-1, 0), (1, 0), (0, -1), (0, 1)):
        intensity[wavevector_harmonic_to_index(harmonic)] = (1/8)*square_depth
    for harmonic in ((-1, 1), (1, -1)):
        intensity[wavevector_harmonic_to_index(harmonic)] = (1/16)*square_depth
    intensity[wavevector_harmonic_to_index((-1, -1))]     = (1/16)*square_depth + (1/4)*stripe_depth * np.exp( 1j * superlattice_phase)
    intensity[wavevector_harmonic_to_index(( 1,  1))]     = (1/16)*square_depth + (1/4)*stripe_depth * np.exp(-1j * superlattice_phase)

    return -intensity

In [7]:
def plot_potential(potential):
    figure = go.Figure(data=[go.Surface(x=position_components, y=position_components, z=-np.abs(np.fft.fftshift(np.fft.fft2(np.fft.fftshift(potential)))))])
    figure.update_layout(autosize=False, width=750, height=750)
    figure.show()

In [8]:
def compute_kinetic():
    effective_wavevectors = np.repeat(quasiwavevectors[:, np.newaxis, :], wavevector_harmonics.shape[0], axis=1) + 2 * wavevector_harmonics
    return np.einsum('kni, kni -> kn', effective_wavevectors, effective_wavevectors).reshape(quasiwavevector_components.size, quasiwavevector_components.size, wavevector_harmonics.shape[0])

In [9]:
@jit(nopython=True)
def compute_transition_energies(kinetic, potential, transition_energies):
    for horizontal_quasiwavevector_index, horizontal_quasiwavector in enumerate(quasiwavevector_components):
        for vertical_quasiwavevector_index, vertical_quasiwavevector in enumerate(quasiwavevector_components):
            transition_energies[horizontal_quasiwavevector_index, vertical_quasiwavevector_index] = np.diag(kinetic[horizontal_quasiwavevector_index, vertical_quasiwavevector_index])

    for this_wavevector_harmonic_index, this_wavevector_harmonic in enumerate(wavevector_harmonics):
        for that_wavevector_harmonic_index, that_wavevector_harmonic in enumerate(wavevector_harmonics):
            potential_index = this_wavevector_harmonic - that_wavevector_harmonic + np.array([max_wavevector_component_harmonic, max_wavevector_component_harmonic])
            if 0 <= potential_index[0] < wavevector_component_harmonics.size and 0 <= potential_index[1] < wavevector_component_harmonics.size:
                potential_energy = potential[potential_index[0], potential_index[1]] 
                transition_energies[:, :, this_wavevector_harmonic_index, that_wavevector_harmonic_index] += potential_energy

In [10]:
@jit(nopython=True)
def compute_energies(transition_energies, energies):
    for horizontal_quasiwavevector_index, _ in enumerate(quasiwavevector_components):
        for vertical_quasiwavevector_index, _ in enumerate(quasiwavevector_components):
            energies[horizontal_quasiwavevector_index][vertical_quasiwavevector_index] = np.linalg.eigvalsh(transition_energies[horizontal_quasiwavevector_index][vertical_quasiwavevector_index])

In [11]:
def plot_energies(energies):
    figure = go.Figure(data=[go.Surface(x=quasiwavevector_components, 
                                        y=quasiwavevector_components, 
                                        z=energies)])
    figure.update_layout(autosize=False, width=750, height=750)
    figure.show()

In [78]:
def compute_tunneling_strengths(energies):
    return np.abs(np.fft.fftshift(np.fft.fft2(np.fft.fftshift(energies[:, :, 0])))) / quasiwavevector_components.size**2

In [106]:
def compute_neighbor_tunneling_strengths(tunneling_strengths, radius=1):
    index_min = quasiwavevector_components.size // 2 - radius
    index_max = quasiwavevector_components.size // 2 + radius
    return tunneling_strengths[index_min:index_max+1, index_min:index_max+1]

# Simulations

In [13]:
RECOIL_ENERGY_752 = 1032 # Hz
RECOIL_ENERGY_532 = 2062 # Hz

## Square

In [109]:
potential = compute_potential(10, 0, 0)
plot_potential(potential)

In [110]:
transition_energies = np.zeros((quasiwavevector_components.size, quasiwavevector_components.size, wavevector_harmonics.shape[0], wavevector_harmonics.shape[0]), dtype=complex)
compute_transition_energies(compute_kinetic(), potential, transition_energies)

In [111]:
energies = np.zeros((quasiwavevector_components.size, quasiwavevector_components.size, wavevector_harmonics.shape[0]))
compute_energies(transition_energies, energies)

In [112]:
plot_energies(energies[:, :, 0])

In [115]:
tunneling_strengths = compute_tunneling_strengths(energies)
neighbor_tunneling_strengths = compute_neighbor_tunneling_strengths(tunneling_strengths, radius=2)
px.imshow(neighbor_tunneling_strengths * RECOIL_ENERGY_752, zmax=32.51314)

## Triangle

In [116]:
potential = compute_potential(4, 8 * RECOIL_ENERGY_532/RECOIL_ENERGY_752, 0)
plot_potential(potential)

In [117]:
transition_energies = np.zeros((quasiwavevector_components.size, quasiwavevector_components.size, wavevector_harmonics.shape[0], wavevector_harmonics.shape[0]), dtype=complex)
compute_transition_energies(compute_kinetic(), potential, transition_energies)

In [118]:
energies = np.zeros((quasiwavevector_components.size, quasiwavevector_components.size, wavevector_harmonics.shape[0]))
compute_energies(transition_energies, energies)

In [119]:
plot_energies(energies[:, :, 0])

In [121]:
tunneling_strengths = compute_tunneling_strengths(energies)
neighbor_tunneling_strengths = compute_neighbor_tunneling_strengths(tunneling_strengths, radius=2)
px.imshow(neighbor_tunneling_strengths * RECOIL_ENERGY_752, zmax=15.83711)